In [1]:
# 爬取豆瓣书评

import pandas as pd
from bs4 import BeautifulSoup
import requests
import random
import time
headers = [
    "Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/64.0.3282.140 Safari/537.36"
    "Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/35.0.1916.153 Safari/537.36",
    "Mozilla/5.0 (Windows NT 6.1; WOW64; rv:30.0) Gecko/20100101 Firefox/30.0",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_2) AppleWebKit/537.75.14 (KHTML, like Gecko) Version/7.0.3 Safari/537.75.14",
    "Mozilla/5.0 (compatible; MSIE 10.0; Windows NT 6.2; Win64; x64; Trident/6.0)",
    'Mozilla/5.0 (Windows; U; Windows NT 5.1; it; rv:1.8.1.11) Gecko/20071127 Firefox/2.0.0.11',
    'Opera/9.25 (Windows NT 5.1; U; en)',
    'Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1; SV1; .NET CLR 1.1.4322; .NET CLR 2.0.50727)',
    'Mozilla/5.0 (compatible; Konqueror/3.5; Linux) KHTML/3.5.5 (like Gecko) (Kubuntu)',
    'Mozilla/5.0 (X11; U; Linux i686; en-US; rv:1.8.0.12) Gecko/20070731 Ubuntu/dapper-security Firefox/1.5.0.12',
    'Lynx/2.8.5rel.1 libwww-FM/2.14 SSL-MM/1.4.1 GNUTLS/1.2.9',
    "Mozilla/5.0 (X11; Linux i686) AppleWebKit/535.7 (KHTML, like Gecko) Ubuntu/11.04 Chromium/16.0.912.77 Chrome/16.0.912.77 Safari/535.7",
    "Mozilla/5.0 (X11; Ubuntu; Linux i686; rv:10.0) Gecko/20100101 Firefox/10.0 "
    ]
with open('./data/book_data/bookID.txt','r') as f:
    ids=f.read().split('\n')
ids=list(set(ids))
while '' in ids:
    ids.remove('')

In [2]:
def get_url_msg(url,headers):

    headers={'User-Agent':random.choice(headers)}
    req= requests.get(url,headers=headers)
    req.raise_for_status()
    req.encoding = 'utf-8'
    soup=BeautifulSoup(req.text,"html.parser")
    reviews=soup.find_all('div',class_='main review-item')
    book=soup.find('div',class_='subject-title').text[3:-1]
    dic={'id':[],'title':[],'score':[],'time':[],'like':[],'dislike':[],'reply':[],'content':[],'book':[]}
    for r in reviews:
        id=r['id']
        title=r.find('h2').text
        time=r.find('span',class_='main-meta').text
        content=r.find('div',class_='short-content').text.strip('\n ')
        
        like=r.find('a',class_='action-btn up').text.strip("\n ")
        if like=="":
            like=0
        else:
            like=int(like)
        
        dislike=r.find('a',class_='action-btn down').text.strip("\n ")
        if dislike=="":
            dislike=0
        else:
            dislike=int(dislike)
        reply=r.find('a',class_='reply').text.strip("回应")
        if reply=="":
            reply=0
        else:
            reply=int(reply)
        if r.find('span',class_='allstar50 main-title-rating')!=None:
            score=5
        elif r.find('span',class_='allstar40 main-title-rating')!=None:
            score=4
        elif r.find('span',class_='allstar30 main-title-rating')!=None:
            score=3
        elif r.find('span',class_='allstar10 main-title-rating')!=None:
            score=1
        elif r.find('span',class_='allstar20 main-title-rating')!=None:
            score=2
        else:
            score = None
        dic['id'].append(id)
        dic['title'].append(title)
        dic['score'].append(score)
        dic['time'].append(time)
        dic['content'].append(content)
        dic['like'].append(like)
        dic['dislike'].append(dislike)
        dic['reply'].append(reply)
        dic['book'].append(book)

    return dic

In [5]:
dic={'id':[],'title':[],'score':[],'time':[],'like':[],'dislike':[],'reply':[],'content':[],'book':[]}

for count,id in enumerate(ids):
    s=time.time()
    for i in range(10):
        url=f"https://book.douban.com/subject/{id}/reviews?start={i*20}"
        dic2=get_url_msg(url,headers)
        for k,v in dic.items():
            dic[k].extend(dic2[k])
        time.sleep(2.3)
    e=time.time()
    e='{:.2f}'.format(e-s)
    print(f"{count}=="+dic['book'][-1]+"=="+ e+"s")
df=pd.DataFrame.from_dict(dic)
with open('./data/book_data/book_reviews.csv','wb')as f:
    df.to_csv(f,sep=',',encoding='utf-8',index=False)

0==钝感力==26.75s
1==置身事内==28.33s
2==追风筝的人==27.22s
3==克拉拉与太阳==26.94s
4==霍乱时期的爱情==26.98s
5==蛤蟆先生去看心理医生==28.25s
6==挪威的森林==27.38s
7==斯通纳==29.43s
8==被讨厌的勇气==27.61s
9==屠夫十字镇==26.35s
10==文城==27.42s
11==百年孤独==27.95s
12==苏东坡传==27.95s
13==一日三秋==27.28s
14==你当像鸟飞往你的山==28.32s
15==下沉年代==28.15s
16==额尔古纳河右岸==27.42s
17==万历十五年==28.15s
18==生死疲劳==27.49s


In [9]:
with open('./data/book_data/book_reviews.csv','rb')as f:
    df=pd.read_csv(f)
print(df.shape)
df.head(10)

(3496, 9)


,id,title,score,time,like,dislike,reply,content,book
0,1168972,读书无用,4.0,2007-06-17 15:00:04,296,21,55,钝感的人看了找到自信，敏感的人看完开始装傻。...,钝感力
1,1207271,这也叫书？！,1.0,2007-09-13 09:27:11,92,6,16,如果这样的书也叫书的话，那样假我以数日，我可...,钝感力
2,1346822,对于某些人来说很亲切的一本书,5.0,2008-04-07 17:13:11,99,25,33,这本书引起这么大争议是我没有想到的。因为我在...,钝感力
3,1195635,敏感还是钝感,3.0,2007-08-18 09:58:15,50,3,13,我从来没有认为“敏感”是个褒义词，相反，“世...,钝感力
4,6321126,没错，生活就是没道理，所以你需要钝感力,5.0,2013-10-10 13:03:22,53,14,8,写过火辣辣的《失乐园》的渡边淳一，...,钝感力
5,2054402,愚笨未必是坏事,5.0,2009-06-03 22:27:55,43,7,11,这本书是我最喜欢的励志书籍之一。 每当我郁闷...,钝感力
6,1765434,作为发明家的渡边淳一,4.0,2009-03-01 14:20:51,36,4,6,渡边淳一有很多身份，为大家所知的有医生和作家...,钝感力
7,3117768,三个字决定一本书的成功,1.0,2010-03-23 23:19:30,26,0,10,最早知道这本书，是看到受君的推荐。说实话，一...,钝感力
8,1480019,自欺欺人的『钝感力』,3.0,2008-08-25 21:28:26,41,20,13,『钝感』是相对于『敏感』而言的新概念。我是冲...,钝感力
9,13628889,太烂了,1.0,2021-06-23 14:49:25,20,0,3,花了几分钟去了解了一些渡边淳一，履历辉煌，但...,钝感力
